In [2]:
import pygame
import random

# 初始化Pygame
pygame.init()

# 設置遊戲視窗大小
WIDTH, HEIGHT = 800, 400
win = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("方塊跑酷遊戲")

# 定義顏色
WHITE = (255, 255, 255)
RED = (255, 0, 0)
BLUE = (0, 0, 255)

# 設置字體和字體大小
font = pygame.font.Font(None, 36)

# 創建方塊角色
class Player:
    def __init__(self):
        self.x = 50
        self.y = HEIGHT - 50
        self.width = 20
        self.height = 20
        self.color = BLUE
        self.vel = 5
        self.score = 0
        self.alive = True

    def move(self, keys):
        if not self.alive:
            return
        if keys[pygame.K_LEFT]:
            self.x -= self.vel
        if keys[pygame.K_RIGHT]:
            self.x += self.vel
        if keys[pygame.K_UP]:
            self.y -= self.vel
        if keys[pygame.K_DOWN]:
            self.y += self.vel

    def draw(self, win):
        pygame.draw.rect(win, self.color, (self.x, self.y, self.width, self.height))

# 創建紅色方塊障礙物
class Obstacle:
    def __init__(self, x, width, height):
        self.x = x
        self.y = HEIGHT - height
        self.width = width
        self.height = height
        self.color = RED
        self.vel = 5

    def move(self):
        self.x -= self.vel

    def draw(self, win):
        pygame.draw.rect(win, self.color, (self.x, self.y, self.width, self.height))

def draw_score(win, score):
    text = font.render(f"Score: {score}", True, (0, 0, 0))
    win.blit(text, (10, 10))

def draw_game_over(win):
    text = font.render("Game Over", True, (255, 0, 0))
    text_rect = text.get_rect(center=(WIDTH // 2, HEIGHT // 2))
    win.blit(text, text_rect)
    restart_text = font.render("Press SPACE or Click to Restart", True, (0, 0, 0))
    restart_rect = restart_text.get_rect(center=(WIDTH // 2, HEIGHT // 2 + 50))
    win.blit(restart_text, restart_rect)

def main():
    player = Player()
    obstacles = []
    clock = pygame.time.Clock()

    running = True
    game_over = False
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        keys = pygame.key.get_pressed()
        if game_over:
            # 如果遊戲結束且按下空白鍵或滑鼠按鈕，重新開始遊戲
            if keys[pygame.K_SPACE] or pygame.mouse.get_pressed()[0]:
                player = Player()
                obstacles = []
                game_over = False
        else:
            # 移動方塊角色
            player.move(keys)

            if player.alive:
                # 創建隨機障礙物
                if random.randrange(0, 100) < 10:
                    obstacle_width = random.randrange(20, 100)
                    obstacle_height = random.randrange(20, 100)
                    obstacles.append(Obstacle(WIDTH, obstacle_width, obstacle_height))

                # 移動和畫出障礙物
                for obstacle in obstacles:
                    obstacle.move()

                # 移除超出畫面的障礙物
                obstacles = [obstacle for obstacle in obstacles if obstacle.x + obstacle.width > 0]

                # 碰撞檢測
                for obstacle in obstacles:
                    if (player.x < obstacle.x + obstacle.width and
                            player.x + player.width > obstacle.x and
                            player.y < obstacle.y + obstacle.height and
                            player.y + player.height > obstacle.y):
                        player.alive = False
                        game_over = True

                # 更新分數
                player.score += 1

        # 繪製遊戲視窗
        win.fill(WHITE)
        draw_score(win, player.score)
        player.draw(win)
        for obstacle in obstacles:
            obstacle.draw(win)
        
        # 如果玩家死亡，顯示 "Game Over" 和 "Press SPACE or Click to Restart"
        if not player.alive:
            draw_game_over(win)

        pygame.display.update()
        clock.tick(30)

    pygame.quit()

if __name__ == "__main__":
    main()